# 00. 환경 검증

## 학습 목표

이 노트북에서는 Istio Demo 환경이 정상적으로 설정되었는지 확인합니다.

**확인 항목:**
1. Kind 클러스터 상태
2. Istio 컴포넌트 상태
3. Observability 스택 (Jaeger, Prometheus, Grafana)
4. 샘플 애플리케이션 상태
5. Sidecar Injection 확인

---

## 사전 조건

이 노트북을 실행하기 전에 다음이 완료되어야 합니다:

1. `00-setup/install.sh` 스크립트 실행 완료
2. `kubectl` 명령어 사용 가능
3. Kind 클러스터 실행 중

In [ ]:
# kubectl 명령어 확인
!kubectl version --client

---

## 1. 클러스터 상태 확인

먼저 Kubernetes 클러스터가 정상적으로 실행 중인지 확인합니다.

In [ ]:
# 현재 사용 중인 Kubernetes context 확인
!kubectl config current-context

In [ ]:
# 클러스터 노드 상태 확인
!kubectl get nodes

**예상 결과:** 노드가 `Ready` 상태여야 합니다.

---

## 2. Istio 컴포넌트 확인

Istio의 핵심 컴포넌트인 `istiod`가 정상적으로 실행 중인지 확인합니다.

**주요 컴포넌트:**
- `istiod`: Istio 컨트롤 플레인 (Pilot, Citadel, Galley 통합)
- `istio-ingressgateway`: 외부 트래픽 진입점

In [ ]:
# istio-system 네임스페이스의 Pod 상태
!kubectl get pods -n istio-system

In [ ]:
# Istio 버전 확인
!kubectl get deployment istiod -n istio-system -o jsonpath='{.spec.template.spec.containers[0].image}'

**예상 결과:** 모든 Pod가 `Running` 상태, `READY` 컬럼이 모두 준비됨.

---

## 3. Observability 스택 확인

분산 추적, 메트릭 수집, 시각화를 위한 도구들을 확인합니다.

| 컴포넌트 | 역할 | 네임스페이스 |
|----------|------|-------------|
| Jaeger | 분산 추적 | observability |
| Prometheus | 메트릭 수집 | monitoring |
| Grafana | 대시보드 | monitoring |
| Kiali | 서비스 메시 토폴로지 | istio-system |

In [ ]:
# Jaeger 상태 확인
print("=== Jaeger ===")
!kubectl get pods -n observability -l app.kubernetes.io/name=jaeger

In [ ]:
# Prometheus & Grafana 상태 확인
print("=== Prometheus & Grafana ===")
!kubectl get pods -n monitoring

**예상 결과:** 모든 Observability 컴포넌트가 `Running` 상태.

---

## 4. 샘플 애플리케이션 확인

테스트에 사용할 샘플 애플리케이션들이 배포되어 있는지 확인합니다.

**주요 애플리케이션:**
- `httpbin`: HTTP 요청/응답 테스트 서비스
- `sleep`: 트래픽 생성용 클라이언트 Pod (curl 포함)

In [ ]:
# istio-demo 네임스페이스의 Pod 상태
!kubectl get pods -n istio-demo

In [ ]:
# 서비스 목록
!kubectl get services -n istio-demo

**예상 결과:** `httpbin`과 `sleep` Pod가 `Running` 상태.

---

## 5. Sidecar Injection 확인

Istio의 핵심 기능인 **자동 사이드카 주입**이 정상 동작하는지 확인합니다.

### Sidecar란?
- 각 Pod에 자동으로 주입되는 **Envoy 프록시**
- 모든 네트워크 트래픽을 가로채어 처리
- 애플리케이션 코드 수정 없이 트래픽 관리, 보안, 관측성 제공

### 확인 방법
1. 네임스페이스에 `istio-injection=enabled` 레이블 확인
2. Pod의 컨테이너 수가 2개인지 확인 (앱 + istio-proxy)

In [ ]:
# 네임스페이스 레이블 확인
!kubectl get namespace istio-demo --show-labels

In [ ]:
# Pod별 컨테이너 확인 (2/2 = sidecar 정상 주입)
!kubectl get pods -n istio-demo -o custom-columns="NAME:.metadata.name,READY:.status.containerStatuses[*].ready,CONTAINERS:.spec.containers[*].name"

**확인 포인트:**
- `READY` 컬럼: `true,true` (두 컨테이너 모두 준비됨)
- `CONTAINERS` 컬럼: 앱 컨테이너 + `istio-proxy`

---

## 연결 테스트

마지막으로 `sleep` Pod에서 `httpbin` 서비스로 요청을 보내 네트워크 연결을 확인합니다.

In [ ]:
# sleep → httpbin 연결 테스트
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s http://httpbin:8000/get | head -10

**예상 결과:** JSON 응답이 정상적으로 반환됨.

---

## 환경 검증 완료!

모든 확인이 통과했다면, Istio 실습 환경이 정상적으로 구성된 것입니다.

### 다음 단계

**[01-관측성.ipynb](./01-관측성.ipynb)** - 분산 추적, 메트릭 수집 등 Istio의 관측성 기능을 학습합니다.

---

### UI 접속 정보

| UI | 포트 | 접속 방법 |
|----|------|----------|
| Kiali | 20001 | http://localhost:20001 |
| Jaeger | 16686 | http://localhost:16686 |
| Grafana | 3000 | http://localhost:3000 (admin/admin) |
| Prometheus | 9090 | http://localhost:9090 |